In [ ]:
# BLOQUE DE LIMPIEZA - EJECUTAR SIEMPRE PRIMERO PARA EVITAR BUG DE CNN (KERNEL SUCIO)
# ===================================================================================

import gc
import tensorflow as tf
from tensorflow import keras

# 1. Limpiar sesión de Keras
keras.backend.clear_session()

# 2. Resetear grafos de TensorFlow
tf.compat.v1.reset_default_graph()

# 3. Liberar memoria GPU (si tienes)
try:
    from numba import cuda
    cuda.select_device(0)
    cuda.close()
except:
    pass

# 4. Forzar recolección de basura
gc.collect()

# 5. NO ELIMINAR EL CACHE!!!

print("Kernel limpiado - listo para entrenar")

Kernel limpiado - listo para entrenar


# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [ ]:
# Step 1.1: Read tic-tac-toe.csv into a dataframe

import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


# Read the CSV file
df = pd.read_csv('/content/drive/MyDrive/ESTUDIOS/IRONHACK/Exercises/Week-15/lab-neural-networks/your-code/tic-tac-toe.csv')

# Display basic information about the dataset
print("Dataset loaded successfully!")
print(f"\nDataset shape: {df.shape}")
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

# Display the first few rows
print("\nFirst 5 rows of the dataset:")
print(df.head())

# Display column names
print("\nColumn names:")
print(df.columns.tolist())

# Display data types
print("\nData types:")
print(df.dtypes)

Dataset loaded successfully!

Dataset shape: (958, 10)
Number of rows: 958
Number of columns: 10

First 5 rows of the dataset:
  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True

Column names:
['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class']

Data types:
TL       object
TM       object
TR       object
ML       object
MM       object
MR       object
BL       object
BM       object
BR       object
class      bool
dtype: object


In [ ]:
# ============================================================================
# Step 1.2: Inspect the dataset - Check for data quality
# ============================================================================

# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())

# Check unique values in each column
print("\nUnique values in each column:")
for col in df.columns:
    print(f"{col}: {df[col].unique()}")

# Check class distribution
print("\nClass distribution:")
print(df['class'].value_counts())


Missing values per column:
TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

Unique values in each column:
TL: ['x' 'o' 'b']
TM: ['x' 'o' 'b']
TR: ['x' 'o' 'b']
ML: ['x' 'o' 'b']
MM: ['o' 'b' 'x']
MR: ['o' 'b' 'x']
BL: ['x' 'o' 'b']
BM: ['o' 'x' 'b']
BR: ['o' 'x' 'b']
class: [ True False]

Class distribution:
class
True     626
False    332
Name: count, dtype: int64


In [ ]:
# ============================================================================
# Step 1.3: Convert categorical values to numeric in all columns
# ============================================================================

# Create a copy to preserve original data
df_numeric = df.copy()

# Define mapping for board positions
# x = 1, o = 0, b (blank) = -1
position_mapping = {'x': 1, 'o': 0, 'b': -1}

# Convert the first 9 columns (board positions) using the mapping
board_columns = ['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR']
for col in board_columns:
    df_numeric[col] = df_numeric[col].map(position_mapping)

# Convert class column (True/False or positive/negative to 1/0)
# True (Player X wins) = 1, False (Player X doesn't win) = 0
df_numeric['class'] = df_numeric['class'].astype(int)

print("\nCategorical mapping applied:")
print(f"Board positions: {position_mapping}")
print(f"Class: True → 1, False → 0")

print("\nFirst 5 rows after conversion:")
print(df_numeric.head())

print("\nData types after conversion:")
print(df_numeric.dtypes)


Categorical mapping applied:
Board positions: {'x': 1, 'o': 0, 'b': -1}
Class: True → 1, False → 0

First 5 rows after conversion:
   TL  TM  TR  ML  MM  MR  BL  BM  BR  class
0   1   1   1   1   0   0   1   0   0      1
1   1   1   1   1   0   0   0   1   0      1
2   1   1   1   1   0   0   0   0   1      1
3   1   1   1   1   0   0   0  -1  -1      1
4   1   1   1   1   0   0  -1   0  -1      1

Data types after conversion:
TL       int64
TM       int64
TR       int64
ML       int64
MM       int64
MR       int64
BL       int64
BM       int64
BR       int64
class    int64
dtype: object


In [ ]:
# ============================================================================
# Step 1.4: Separate inputs and output
# ============================================================================

# Separate features (X) and target (y)
X = df_numeric[board_columns].values  # Input features (9 board positions)
y = df_numeric['class'].values        # Output (game result)

print(f"\nInput shape (X): {X.shape}")
print(f"Output shape (y): {y.shape}")

print(f"\nFirst 3 samples of X (inputs):")
print(X[:3])

print(f"\nFirst 3 samples of y (outputs):")
print(y[:3])


Input shape (X): (958, 9)
Output shape (y): (958,)

First 3 samples of X (inputs):
[[1 1 1 1 0 0 1 0 0]
 [1 1 1 1 0 0 0 1 0]
 [1 1 1 1 0 0 0 0 1]]

First 3 samples of y (outputs):
[1 1 1]


In [ ]:
# ============================================================================
# Step 1.5: Normalize the input data
# ============================================================================

# Normalize X from [-1, 1] to [0, 1] using Min-Max scaling
# Formula: X_normalized = (X - X_min) / (X_max - X_min)
X_min = X.min()
X_max = X.max()

X_normalized = (X - X_min) / (X_max - X_min)

print(f"\nOriginal data range: [{X_min}, {X_max}]")
print(f"Normalized data range: [{X_normalized.min()}, {X_normalized.max()}]")

print(f"\nFirst 3 samples of X_normalized:")
print(X_normalized[:3])

print(f"\nSummary:")
print(f"  • Total samples: {X_normalized.shape[0]}")
print(f"  • Input features: {X_normalized.shape[1]}")
print(f"  • Positive cases (X wins): {(y == 1).sum()}")
print(f"  • Negative cases (X loses): {(y == 0).sum()}")
print(f"\nData is ready for neural network training!")


Original data range: [-1, 1]
Normalized data range: [0.0, 1.0]

First 3 samples of X_normalized:
[[1.  1.  1.  1.  0.5 0.5 1.  0.5 0.5]
 [1.  1.  1.  1.  0.5 0.5 0.5 1.  0.5]
 [1.  1.  1.  1.  0.5 0.5 0.5 0.5 1. ]]

Summary:
  • Total samples: 958
  • Input features: 9
  • Positive cases (X wins): 626
  • Negative cases (X loses): 332

Data is ready for neural network training!


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [ ]:
# STEP 2: BUILD NEURAL NETWORK

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import optimizers

# ============================================================================
# Step 2.1: Split the training and test data
# ============================================================================

# Split data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Ensures equal distribution of classes
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"\nTraining set shape - X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Test set shape - X_test: {X_test.shape}, y_test: {y_test.shape}")

# Check class distribution
print(f"\nTraining set class distribution:")
print(f"  Class 1 (X wins): {(y_train == 1).sum()} ({(y_train == 1).sum()/len(y_train)*100:.1f}%)")
print(f"  Class 0 (X loses/draw): {(y_train == 0).sum()} ({(y_train == 0).sum()/len(y_train)*100:.1f}%)")



Training set size: 766 samples
Test set size: 192 samples

Training set shape - X_train: (766, 9), y_train: (766,)
Test set shape - X_test: (192, 9), y_test: (192,)

Training set class distribution:
  Class 1 (X wins): 501 (65.4%)
  Class 0 (X loses/draw): 265 (34.6%)


In [ ]:
# ============================================================================
# Step 2.2: Create a Sequential model
# ============================================================================

model = Sequential()

print("✓ Sequential model created")

# ============================================================================
# Step 2.3: Add layers to the model
# ============================================================================

# Input layer + First hidden layer
model.add(tf.keras.layers.Input(shape=(9,), name='input_layer'))
model.add(tf.keras.layers.Dense(64, activation='relu', name='hidden_layer_0')) # Added a Dense layer after Input

# Second hidden layer
model.add(tf.keras.layers.Dense(32, activation='relu', name='hidden_layer_1'))

# Third hidden layer
model.add(tf.keras.layers.Dense(16, activation='relu', name='hidden_layer_2'))

# Output layer - 2 classes (Win or Not Win), using softmax
model.add(tf.keras.layers.Dense(2, activation='softmax', name='output_layer'))

print("Layers added:")
print("  1. Input(shape=(9,)) - Input layer")
print("  2. Dense(64, activation='relu') - Hidden layer 0 (formerly Input layer description)")
print("  3. Dense(32, activation='relu') - Hidden layer 1")
print("  4. Dense(16, activation='relu') - Hidden layer 2")
print("  5. Dense(2, activation='softmax') - Output layer")

# Display model architecture
print("\n" + "-"*60)
print("Model Architecture Summary:")
print("-"*60)
model.summary()

✓ Sequential model created
Layers added:
  1. Input(shape=(9,)) - Input layer
  2. Dense(64, activation='relu') - Hidden layer 0 (formerly Input layer description)
  3. Dense(32, activation='relu') - Hidden layer 1
  4. Dense(16, activation='relu') - Hidden layer 2
  5. Dense(2, activation='softmax') - Output layer

------------------------------------------------------------
Model Architecture Summary:
------------------------------------------------------------


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hidden_layer_0 (Dense)          │ (None, 64)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_1 (Dense)          │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden_layer_2 (Dense)          │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,282 (12.82 KB)

 Trainable params: 3,282 (12.82 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ============================================================================
# Step 2.4: Compile the model
# ============================================================================

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("✓ Model compiled successfully")
print("  Optimizer: adam")
print("  Loss function: sparse_categorical_crossentropy")
print("  Metrics: accuracy")



✓ Model compiled successfully
  Optimizer: adam
  Loss function: sparse_categorical_crossentropy
  Metrics: accuracy


In [ ]:
# ============================================================================
# Step 2.5: Fit the training data
# ============================================================================

print("\n" + "="*60)
print("Step 2.5: Training the model")
print("-"*60)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,  # Use 20% of training data for validation
    verbose=1
)

print("\n✓ Model training completed!")




Step 2.5: Training the model
------------------------------------------------------------
Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.6548 - loss: 0.6620 - val_accuracy: 0.6364 - val_loss: 0.6550
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6590 - loss: 0.6370 - val_accuracy: 0.6364 - val_loss: 0.6447
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6584 - loss: 0.6223 - val_accuracy: 0.6364 - val_loss: 0.6371
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6679 - loss: 0.5990 - val_accuracy: 0.6364 - val_loss: 0.6287
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6998 - loss: 0.6039 - val_accuracy: 0.6429 - val_loss: 0.6262
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7070 - loss: 0.5630 - val_accuracy: 0.6818 - val_loss: 0.6154
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7391 - loss: 0.5650 - val_accuracy: 0.6753 - val_loss: 0.6115
Epoch 8/50
20/20 ━━━━━━━━━

In [ ]:
# ============================================================================
# Step 2.6: Evaluate the model with test data
# ============================================================================

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f"\nTest Results:")
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Make predictions on test data
y_pred_probs = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate additional metrics
from sklearn.metrics import classification_report, confusion_matrix

print("\n" + "-"*60)
print("Detailed Classification Report:")
print("-"*60)
print(classification_report(y_test, y_pred, target_names=['X Loses/Draw', 'X Wins']))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("Format: [[True Negatives, False Positives]")
print("         [False Negatives, True Positives]]")




Test Results:
  Loss: 0.3953
  Accuracy: 0.8438 (84.38%)

------------------------------------------------------------
Detailed Classification Report:
------------------------------------------------------------
              precision    recall  f1-score   support

X Loses/Draw       0.85      0.67      0.75        67
      X Wins       0.84      0.94      0.89       125

    accuracy                           0.84       192
   macro avg       0.85      0.80      0.82       192
weighted avg       0.84      0.84      0.84       192


Confusion Matrix:
[[ 45  22]
 [  8 117]]
Format: [[True Negatives, False Positives]
         [False Negatives, True Positives]]


In [ ]:
#Saving the Model

model.save("tic_tac_toe_model.h5")

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
#Loading the model

from tensorflow.keras.models import load_model

model = load_model("tic_tac_toe_model.h5")

In [ ]:
import numpy as np

# Select random indices
random_indices = np.random.choice(len(X_test), size=10, replace=False)

X_sample = X_test[random_indices]
y_true = y_test[random_indices]

In [ ]:
y_pred_probs = model.predict(X_sample)
y_pred = np.argmax(y_pred_probs, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


In [ ]:
labels = ['X Loses/Draw', 'X Wins']

for i in range(len(X_sample)):
    print(f"Sample {i+1}")
    print(f"  True label: {labels[y_true[i]]}")
    print(f"  Predicted:  {labels[y_pred[i]]}")
    print(f"  Confidence: {np.max(y_pred_probs[i])*100:.2f}%")
    print("-"*40)

Sample 1
  True label: X Wins
  Predicted:  X Wins
  Confidence: 93.70%
----------------------------------------
Sample 2
  True label: X Loses/Draw
  Predicted:  X Loses/Draw
  Confidence: 94.02%
----------------------------------------
Sample 3
  True label: X Wins
  Predicted:  X Wins
  Confidence: 87.75%
----------------------------------------
Sample 4
  True label: X Loses/Draw
  Predicted:  X Loses/Draw
  Confidence: 86.10%
----------------------------------------
Sample 5
  True label: X Wins
  Predicted:  X Wins
  Confidence: 99.17%
----------------------------------------
Sample 6
  True label: X Wins
  Predicted:  X Loses/Draw
  Confidence: 72.80%
----------------------------------------
Sample 7
  True label: X Wins
  Predicted:  X Wins
  Confidence: 89.61%
----------------------------------------
Sample 8
  True label: X Wins
  Predicted:  X Wins
  Confidence: 97.23%
----------------------------------------
Sample 9
  True label: X Loses/Draw
  Predicted:  X Loses/Draw
  C

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# =========================================
# STEP 4 — IMPROVE YOUR MODEL (FULL VERSION)
# =========================================

import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# -----------------------------------------
# 1. BUILD A BETTER MODEL (MORE LAYERS)
# -----------------------------------------

model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),  # Helps prevent overfitting

    Dense(128, activation='relu'),
    Dropout(0.3),

    Dense(64, activation='relu'),
    Dropout(0.3),

    Dense(32, activation='relu'),

    Dense(2, activation='softmax')  # 2 output classes: X Loses/Draw, X Wins
])

# -----------------------------------------
# 2. COMPILE WITH CUSTOM LEARNING RATE
# -----------------------------------------

optimizer = Adam(learning_rate=0.001)  # Try 0.0005 or 0.0001 if needed

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# -----------------------------------------
# 3. TRAIN WITH MORE EPOCHS
# -----------------------------------------

history = model.fit(
    X_train,
    y_train,
    epochs=200,          # More epochs = more learning (until plateau)
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# -----------------------------------------
# 4. EVALUATE ON TEST DATA
# -----------------------------------------

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print("\nTest Results:")
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_accuracy*100:.2f}%")

# -----------------------------------------
# 6. SAVE THE IMPROVED MODEL
# -----------------------------------------

model.save("tic_tac_toe_improved_model.h5")
print("\nModel saved as 'tic_tac_toe_improved_model.h5'")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.6103 - loss: 0.6648 - val_accuracy: 0.6364 - val_loss: 0.6501
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6769 - loss: 0.6205 - val_accuracy: 0.6364 - val_loss: 0.6479
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6393 - loss: 0.6329 - val_accuracy: 0.6364 - val_loss: 0.6371
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6508 - loss: 0.6192 - val_accuracy: 0.6234 - val_loss: 0.6411
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6715 - loss: 0.6074 - val_accuracy: 0.6753 - val_loss: 0.6223
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7017 - loss: 0.5912 - val_accuracy: 0.6623 - val_loss: 0.6196
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6879 - loss: 0.5740 - val_accuracy: 0.6948 - val_loss: 0.6102
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7279 - loss: 0.5503 - val_accuracy: 0.6753 - 


Test Results:
  Loss: 0.2869
  Accuracy: 91.15%

Model saved as 'tic_tac_toe_improved_model.h5'


In [ ]:
#Loading the model

from tensorflow.keras.models import load_model

model = load_model("tic_tac_toe_improved_model.h5")

In [ ]:
# -----------------------------------------
# 5. MAKE PREDICTIONS
# -----------------------------------------

y_pred_probs = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred, target_names=['X Loses/Draw', 'X Wins']))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Detailed Classification Report:
              precision    recall  f1-score   support

X Loses/Draw       0.98      0.76      0.86        67
      X Wins       0.89      0.99      0.94       125

    accuracy                           0.91       192
   macro avg       0.93      0.88      0.90       192
weighted avg       0.92      0.91      0.91       192


Confusion Matrix:
[[ 51  16]
 [  1 124]]


**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# Adding layers to my neural network
# Lowering the learning rate from my adam model when compiling
# More epochs when model.fit